In [ ]:
### THIS IS THE SAME AS WHAT WAS DONE FOR ALL OF THE OTHER DATA in 1_cleaning_health_data
###... DC HAD TO BE DONE SEPARATELY BECAUSE IT IS NOT WITHIN A COUNTY
###... make sure you've read through the 1_cleaning_health_data notebook if you dont understand a step

import pandas as pd
import regex as re
import addfips # Must manaully add fips data to DC data
from progressbar import ProgressBar

In [ ]:
df = pd.read_csv("./datasets/500_Cities__Local_Data_for_Better_Health__2017_release.csv")

In [ ]:
dc = df[df["StateAbbr"] == "DC"]

In [ ]:
dc_census = dc[dc['GeographicLevel'] == "Census Tract"]

In [ ]:
re.findall("-\d+.\d+", dc_census.GeoLocation[230262])[0] # this is how the regex will work for Long

In [ ]:
dc_census['Latitude'] = [float(re.findall("(\d+.\d+),", dc_census.GeoLocation[i])[0]) for i in dc_census.index] # regex for lat

In [ ]:
dc_census['Longitude'] = [float(re.findall("-\d+.\d+", dc_census.GeoLocation[i])[0]) for i in dc_census.index] # regex for long

In [ ]:
dc_census.Measure.value_counts()

In [ ]:
dc_census.Measure.value_counts().index[0]

In [ ]:
measures_of_interest = df.Measure.value_counts().index

In [ ]:
dc_df_list = [dc_census[dc_census["Measure"] == x] for x in measures_of_interest] # making a df for each measure

In [ ]:
metric_dfs = []
for df in dc_df_list:
    df.reset_index(inplace=True, drop=True)
    temp = pd.DataFrame({df.MeasureId[0]:df.Data_Value, "UniqueID":df.UniqueID,
                                       "geometry":df.GeoLocation, "state":df.StateAbbr,
                            "pop":df.PopulationCount, "longitude": df.Longitude,
                           "latitude":df.Latitude})
    metric_dfs.append(temp)

In [ ]:
metric_dfs[0].UniqueID[3][0:5]

In [ ]:
metric_dfs[0].UniqueID.value_counts().sum()

In [ ]:
master = pd.merge(metric_dfs[0],metric_dfs[1], how='left',on="UniqueID")
for i in metric_dfs[2:]:
    master = master.merge(i, how = "left",on="UniqueID")

In [ ]:
master.columns

In [ ]:
master = master.T.drop_duplicates().T

In [ ]:
master.columns

In [ ]:
master.columns

In [ ]:
master.rename({"geometry_y":"geometry","state_y":"state","pop_y":"population","longitude_y":"longitude",
              "latitude_y":"latitude"}, axis = 1, inplace = True)
master.drop(['geometry_x', 'state_x', 'pop_x', 'longitude_x','latitude_x'], axis = 1, inplace = True)

In [ ]:
master.dropna().shape, master.shape

In [ ]:
master.dropna(inplace=True)

In [ ]:
master['fips'] = [x[0:5] for x in master.UniqueID]

In [ ]:
master.head()

In [ ]:
master.columns

In [ ]:
agg_cols = ['DENTAL','SLEEP', 'BPHIGH', 'COPD', 'CANCER', 'CHECKUP',
       'DIABETES', 'CSMOKING', 'CHOLSCREEN', 'PHLTH', 'STROKE', 'KIDNEY',
       'CHD', 'HIGHCHOL', 'CASTHMA', 'BINGE', 'LPA', 'BPMED', 'ARTHRITIS',
       'MHLTH', 'OBESITY', 'ACCESS2', 'PAPTEST', 'COLON_SCREEN', 'MAMMOUSE',
       'TEETHLOST','COREM', 'COREW']

In [ ]:
clean_dc = master[agg_cols]

In [ ]:
for col in clean_dc.dtypes.index:
    clean_dc[col] = clean_dc[col].astype(float)
clean_dc["fips"] = clean_dc["fips"].astype(int).astype(str)

In [ ]:
# All entries here share the same CITY fips code (different from county)
# For our analysis, we will treat DC as a county and aggregate its data
dc_agg = clean_dc.groupby("fips").mean().reset_index(drop = False)

#We then need to assign the aggregated data the correct DC COUNTY fips code = 11001
dc_agg["fips"] = "11001"

In [ ]:
dc_agg.head()

In [ ]:
health_df = pd.read_csv("./datasets/agg_county_data_final.csv", dtype={"fips":str})
health_df.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
#These are the columns we filtered to in the first notebook because they had values for each row
cols = list(health_df.columns)

In [ ]:
health_df.columns

In [ ]:
dc_agg = dc_agg[cols]

In [ ]:
dc_agg.head()

In [ ]:
health_df = pd.concat([health_df, dc_agg])


In [ ]:
health_df.tail()

In [ ]:
health_df.reset_index(drop=True, inplace=True)

In [ ]:
health_df.to_csv("./datasets/all_health_data_final.csv")